In [3]:
import pandas as pd
import ast

# 파일 경로
retry_success_path = "유통공사_retry_success_20250708_084350.csv"
template_path = "유통공사_도매시장_배추_20180103-20241231.csv"
output_path = "유통공사_retry_success_형식변환_20250708.csv"

# CSV 파일 불러오기 (한글 포함이므로 cp949 인코딩)
df_retry_success = pd.read_csv(retry_success_path, encoding='cp949')
df_template = pd.read_csv(template_path, encoding='cp949')

# "0" 값만 있는 행 제거
df_filtered = df_retry_success[df_retry_success["0"] != "0"].copy()
df_filtered.columns = ["raw"]  # 열 이름 재지정

# 문자열을 파이썬 딕셔너리로 안전하게 변환
df_parsed = df_filtered["raw"].apply(ast.literal_eval)
df_data = pd.DataFrame(df_parsed.tolist())

# 열 순서 맞추기 (template 기준)
df_data = df_data.reindex(columns=df_template.columns)

# 저장
df_data.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11992\857768080.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_template = pd.read_csv(template_path, encoding='cp949')


In [4]:
# 병합하기

import pandas as pd

# 파일 경로 설정
existing_path = "유통공사_도매시장_배추_20180103-20241231.csv"
new_path = "유통공사_retry_success_형식변환_20250708.csv"
output_path = "유통공사_도매시장_배추_병합중복제거_20250708.csv"

# CSV 파일 불러오기
df_existing = pd.read_csv(existing_path, encoding='cp949')
df_new = pd.read_csv(new_path, encoding='cp949')

# 데이터 병합
df_merged = pd.concat([df_existing, df_new], ignore_index=True)

# 날짜 컬럼을 datetime으로 변환 후 정렬
df_merged["trd_clcln_ymd"] = pd.to_datetime(df_merged["trd_clcln_ymd"])
df_merged.sort_values("trd_clcln_ymd", inplace=True)

# 완전 중복 행 제거
df_merged.drop_duplicates(inplace=True)

# 저장
df_merged.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11992\379043658.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_existing = pd.read_csv(existing_path, encoding='cp949')
C:\Users\Admin\AppData\Local\Temp\ipykernel_11992\379043658.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv(new_path, encoding='cp949')


In [8]:
import pandas as pd

# 파일 경로
input_path = "유통공사_도매시장_배추_병합중복제거_20250708.csv"
output_path = "유통공사_도매시장_배추_한글컬럼명_20250708.csv"

# CSV 불러오기
df = pd.read_csv(input_path, encoding='cp949')

# 새로운 한글 컬럼명
new_columns = [
    '평균가격', '법인코드', '법인이름', '상품 대분류 코드', '상품 대분류 이름',
    '상품 중분류 코드', '상품 중분류 이름', '상품 소분류 코드', '상품 소분류 이름',
    '등급코드', '등급이름', '최고가', '최저가', '포장코드', '포장이름',
    '산지코드', '산지이름', '크기코드', '크기이름', '총가격', '날짜',
    '매매구분', '단위코드', '단위', '단위물량', '단위총물량', '도매시장코드', '도매시장이름'
]

# 컬럼명 변경
if len(df.columns) == len(new_columns):
    df.columns = new_columns
else:
    raise ValueError("컬럼 수가 맞지 않아 컬럼명을 변경할 수 없습니다.")

# 저장
df.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11992\436796294.py:8: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, encoding='cp949')


In [10]:
import pandas as pd

# 파일 경로
input_path = "유통공사_도매시장_배추_한글컬럼명_20250708.csv"
output_path = "유통공사_배추_요약데이터_20250708.csv"

# CSV 파일 불러오기
df = pd.read_csv(input_path, encoding='cp949')

selected_columns = ['날짜', '상품 중분류 이름', '등급이름', '총가격', '단위총물량', '산지코드', '산지이름']
df_selected = df[selected_columns]

# 새 CSV로 저장
df_selected.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11992\772746532.py:8: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, encoding='cp949')


In [20]:
import pandas as pd

# 파일 경로
base_file = "유통공사_배추_요약데이터_20250708.csv"
jigpam_code_file = "산지코드_직팜.csv"

# 파일 읽기 (cp949 인코딩)
df_base = pd.read_csv(base_file, encoding='cp949')
df_code = pd.read_csv(jigpam_code_file, encoding='cp949')

# '산지코드'를 숫자형으로 변환
df_code['산지코드'] = pd.to_numeric(df_code['산지코드'], errors='coerce')

# 800000 이상인 항목만 필터링하여 매핑 딕셔너리 생성
code_map = df_code[df_code['산지코드'] >= 800000] \
              .set_index('산지이름')['산지코드'].to_dict()

# 안전한 대체 함수 정의
def safe_replace(row):
    new_code = code_map.get(row['산지이름'], None)
    if new_code is not None:
        return new_code
    return row['산지코드']  # 매핑 안되면 기존 값 유지

# 정제 후 문자형으로 저장
df_base['산지코드'] = df_base['산지코드'].apply(clean_origin_code)

# 결과 저장
output_path = "유통공사_배추_요약데이터_산지이름기반수정.csv"
df_base.to_csv(output_path, index=False, encoding='cp949')

print(f"✅ 산지이름 기반으로 산지코드 수정 완료: {output_path}")

✅ 산지이름 기반으로 산지코드 수정 완료: 유통공사_배추_요약데이터_산지이름기반수정.csv


In [21]:
import pandas as pd
import re

# CSV 파일 로드
df = pd.read_csv("유통공사_배추_요약데이터_산지이름기반수정.csv", encoding = 'cp949')

# 산지코드 정제 함수
def clean_origin_code(code):
    cleaned = re.sub(r'\D', '0', str(code))     # 숫자가 아닌 문자는 0으로 치환
    cleaned = cleaned.ljust(6, '0')              # 6자리로 패딩
    return cleaned

# '산지코드' 컬럼 정제 후 덮어쓰기
df['산지코드'] = df['산지코드'].apply(clean_origin_code)

# 저장
output_path = "유통공사_배추_요약데이터_정제.csv"
df.to_csv(output_path, index=False, encoding = 'cp949')

print(f"정제된 데이터가 저장되었습니다: {output_path}")

정제된 데이터가 저장되었습니다: 유통공사_배추_요약데이터_정제.csv
